In [7]:
#test which virtualenv running in
import sys
sys.prefix

'/usr'

In [8]:
import ipywidgets as widgets

In [1]:
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage

from config import TRANSFORMED_TRAIN_ANNOTATIONS_PATH,TRANSFORMED_VALIDATION_ANNOTATIONS_PATH,IMAGE_SIZE
from models.six_stage_linear_model import ModelMaker
import dataset_functions
import visualizations as v

In [10]:
tf.version.VERSION

'2.1.0-dev20191124'

In [11]:
#Training config, can be moved later to main config
CACHE=False
CACHE_RAMFS=False #uses a ramfs file to force using main memory
BATCH_SIZE=32  #must be small if caching
SHUFFLE=True
PREFETCH=10  #size of prefetch size, 0 to disable

In [12]:
tpu_ip='10.0.3.2'
# !!!MAKE SURE THE TPU ADDRESS IS CORRECT!!
# 1.ip must be correct
# 2.tpu must be turned on!
# 3.version must be 'nightly-2.x'
# 4.tpu must be reachable (check with gce netowrking/connectivity test)
# if not this will hang
tpu_address = 'grpc://'+tpu_ip+':8470'
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: 10.0.3.2:8470


INFO:tensorflow:Initializing the TPU system: 10.0.3.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


---
Caching using ramfs (irrelevant for tpu)

In [13]:
if CACHE_RAMFS:
    !mkdir /tmp/ramdisk
    !sudo umount /tmp/ramdisk
    !sudo mount -t ramfs -o size=512m ramfs /tmp/ramdisk
    !sudo chown $LOGNAME:$LOGNAME /tmp/ramdisk

In [14]:
if CACHE_RAMFS:
    cache_loc="/tmp/ramdisk/cache_t"
    cache_v_loc="/tmp/ramdisk/cache_v"
else:
    cache_loc=None
    cache_v_loc=None

In [15]:
cache_loc

---
# Make dataset

In [16]:
label_transformer=dataset_functions.LabelTransformer()
@tf.function
def make_label_tensors(elem):
    """Transforms a dict data element:
    1.Read jpg to tensor 
    1.1 Resize img to correct size for network
    2.Convert keypoints to correct form label tensor
    3.Convert joints to correct form label tensor
    outputs a tuple data element"""
    
    idd=elem['id']
    kpt_tr=label_transformer.keypoints_spots_vmapfn(elem['kpts'])
    paf_tr=label_transformer.joints_PAFs(elem['joints'])
    
    image_raw=elem["image_raw"]
    image=tf.image.decode_jpeg(image_raw,channels=3)
    image=tf.image.convert_image_dtype(image,dtype=tf.float32)
    image=tf.image.resize(image,IMAGE_SIZE)
    return image,(paf_tr,kpt_tr),idd

In [17]:
@tf.function
def place_training_labels(image,labels,idd):
    """Disterbutes labels into the correct configuration for the model, ie 4 PAF stage, 2 kpt stages
    must match the model"""
    paf_tr=labels[0]
    kpt_tr=labels[1]
    return image,(paf_tr,paf_tr,paf_tr,paf_tr,kpt_tr,kpt_tr) #this should match the model outputs, and is different for each model

Read and Parse the TFrecords

In [18]:
DATASET_SIZE=56000 #exact size not critical
DATASET_VAL_SIZE=2500 

----
figure out GCS storage


In [19]:
train_prefix=TRANSFORMED_TRAIN_ANNOTATIONS_PATH.split(os.sep)[-1]
val_prefix=TRANSFORMED_VALIDATION_ANNOTATIONS_PATH.split(os.sep)[-1]

In [20]:
bucket_name="datasets_bucket_a"
gs_prefix="gs://"

In [21]:
storage_client = storage.Client() #must have apropriate authenitication to work 

train_blobs = storage_client.list_blobs(bucket_name,prefix=train_prefix)
val_blobs = storage_client.list_blobs(bucket_name,prefix=val_prefix)

In [22]:
tfrecord_files_train=[gs_prefix+bucket_name+'/'+blob.name for blob in train_blobs]
tfrecord_files_val=[gs_prefix+bucket_name+'/'+blob.name for blob in val_blobs]

In [23]:
#tfrecord_files_train

In [24]:
ds = tf.data.TFRecordDataset(tfrecord_files_train)

In [25]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds = tf.data.TFRecordDataset(tfrecord_files_train) #numf reads can be put here, but I don't think I/O is the bottleneck

#raw format to imgs,tensors(coords kpts)
ds=ds.map(TF_parser.read_tfrecord)

#cache  ,caching is here before decompressing jpgs and label tensors (should be ~9GB) , (full dataset should be ~90, cache later if RAM aviable)
if CACHE: ds=ds.cache(cache_loc)
if SHUFFLE: ds=ds.shuffle(100)    
    
#Augmentation should be here, to operate on smaller tensors
    
#imgs,tensors to label_tensors (46,46,17/38)
ds=ds.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds=ds.map(place_training_labels) 

#batch
ds=ds.batch(BATCH_SIZE)
#repeat
ds=ds.repeat()
#prefetch
if PREFETCH: ds=ds.prefetch(PREFETCH)

Make validation dataset

In [26]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds_v = tf.data.TFRecordDataset(tfrecord_files_val) #numf reads can be put here, but I don't think I/O is the bottleneck
#raw format to imgs,tensors(coords kpts)
ds_v=ds_v.map(TF_parser.read_tfrecord)   

#cache  
if CACHE: ds_v=ds_v.cache(cache_v_loc)
    
#imgs,tensors to label_tensors (46,46,17/38)
ds_v=ds_v.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds_v=ds_v.map(place_training_labels) 
#batch
ds_v=ds_v.batch(BATCH_SIZE)


Examine datasets

In [27]:
st=next(iter(ds))
#st
#st_v=next(iter(ds_v))
#v.show_pafs_kpts_img()

In [28]:
len(st[0])

32

In [29]:
ds

<PrefetchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [30]:
ds_v

<BatchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [31]:
#st[0]

---
# Callbacks

In [40]:
#local dir for CPU/GPU training
#checkpoints_dir='./checkpoints'
#!mkdir checkpoints
#gcs dir for TPU training
checkpoints_dir='gs://dl_checkpoints'

In [49]:
import datetime
t=datetime.datetime.now()
now=t.strftime("%a%d%m%y-%H%M")
now

'Tue261119-2108'

In [50]:
checkpoints_path = checkpoints_dir+"/ModelWeights-"+now+"-{epoch:04d}.ckpt" 
checkpoints_path

'gs://dl_checkpoints/ModelWeights-Tue261119-2108-{epoch:04d}.ckpt'

In [63]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {checkpoints_dir}/test
!gsutil rm {checkpoints_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_checkpoints/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [51]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                             save_weights_only=True,
                                             verbose=1)

---
# Model

In [37]:
@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [38]:
with strategy.scope():
    model_maker=ModelMaker()
    train_model,test_model=model_maker.create_models()
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=mse_2d_loss
                    ,metrics=["acc"]                    
                   )


## Load weights

In [57]:
#load latest weights
latest = tf.train.latest_checkpoint(checkpoints_dir)
if latest:
    print("Found checkpoint: %s" % latest)

Found checkpoint: gs://dl_checkpoints/ModelWeights-Tue:26:11:19-21:03-0002.ckpt


In [59]:
load=widgets.Checkbox(
    value=False,
    description='Load last weights'
)
load

Checkbox(value=False, description='Load last weights')

In [60]:
if latest and (load.value):
    train_model.load_weights(latest)

---
Actually training

In [46]:
steps_per_epoch=int(DATASET_SIZE/BATCH_SIZE)

In [47]:
train_model.fit(
    ds,epochs=2
    ,steps_per_epoch=7
    ,validation_data=ds_v
    ,callbacks=[cp_callback]
)

Train for 7 steps
Epoch 1/2


6/7 [========================>.....] - ETA: 10s - loss: 0.1549 - stage1paf_output_loss: 0.0083 - stage2paf_output_loss: 0.0097 - stage3paf_output_loss: 0.0091 - stage4paf_output_loss: 0.0122 - stage5heatmap_output_loss: 0.0490 - stage6heatmap_output_loss: 0.0667 - stage1paf_output_acc: 0.0091 - stage2paf_output_acc: 0.1919 - stage3paf_output_acc: 0.1335 - stage4paf_output_acc: 0.0022 - stage5heatmap_output_acc: 0.0463 - stage6heatmap_output_acc: 0.0458WARNING:tensorflow:5 out of the last 13 calls to <function mse_2d_loss at 0x7f2cf3515d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details


Epoch 00001: saving model to gs://dl_checkpoints/ModelWeights-Tue:26:11:19-21:03-0001.ckpt
7/7 [==============================] - 110s 16s/step - loss: 0.1468 - stage1paf_output_loss: 0.0072 - stage2paf_output_loss: 0.0085 - stage3paf_output_loss: 0.0079 - stage4paf_output_loss: 0.0106 - stage5heatmap_output_loss: 0.0487 - stage6heatmap_output_loss: 0.0640 - stage1paf_output_acc: 0.0112 - stage2paf_output_acc: 0.2882 - stage3paf_output_acc: 0.2333 - stage4paf_output_acc: 0.1182 - stage5heatmap_output_acc: 0.0467 - stage6heatmap_output_acc: 0.0456 - val_loss: 0.0839 - val_stage1paf_output_loss: 7.7557e-04 - val_stage2paf_output_loss: 7.7591e-04 - val_stage3paf_output_loss: 7.7590e-04 - val_stage4paf_output_loss: 7.7603e-04 - val_stage5heatmap_output_loss: 0.0393 - val_stage6heatmap_output_loss: 0.0414 - val_stage1paf_output_acc: 0.4582 - val_stage2paf_output_acc: 0.8990 - val_stage3paf_output_acc: 0.8930 - val_stage4paf_output_acc: 0.9000 - val_stage5heatmap_output_acc: 0.0416 - val_st

In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data

# !!!!!!!IT IS FUCKING WORKING!!!!!!!

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Lambda(lambda x: x, input_shape=(10,10,5)))

model.compile(loss="mse")


In [6]:
model.predict(tf.zeros((1,10,10,5)))

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
  

In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state([1, 2, 3, 4], [0, 2, 3, 4])